In [ ]:
import random

In [ ]:
def objective_function(vector):
    '''
    Description:
        This function will calculate the 'cost' for the given vector. The cost is simply the sum of the squares 
        of the individual vectors

    Input
        vector (list): a list of two floats for the cost calculation

    Output:
        (Float): sum of the squares of the two vectors
    '''
    return sum([i**2 for i in vector])

def rand_in_bounds(minimum, maximum):
    '''
    Input: 
        minimum (float): lower bounds of the vector
        maximum (float): upper bounds of the vectors

    Output:
        (float): Cost of the two vectors
    '''
    return minimum + ((maximum-minimum)*random.random())

def random_vector(minmax):
    '''
    Description:
        This function will take a series of bounds and generate a random number 

    Input:
        minmax (List): list of the upper and lower bounds for the function (optimization window)

    Output:
        (List): List of two floats. Candidates for the function optimization. 
    '''
    return [rand_in_bounds(i[0], i[1]) for i in minmax]

def large_step_size(iters, step_size, s_factor, l_factor, iter_mult):
    ''''
    Description:
        Determine how large of a step to take when looking for new solutions. Every `iter_mut` iterations,
        the algorithm takes a large step.

    Input:
        iters (int): Number iteration number
        step_size (float): The base step amount
        s_factor (float): The multiplier of the normal step size for a small step
        l_factor (flaot): The multiplier of the normal step size for a large step
        iter_mult (int) : How often to take a large step

    Output:
        (float): how large of a step to take for the particuar iteration
    ''''
    if (iters > 0) and (iters % iter_mult == 0):
        return step_size * l_factor
    else:
        return step_size * s_factor

def take_step(minmax, current, step_size):
    '''
    Take a step from the current position

    Input:
        minmax (list): The bounds for the algorithm
        current (list): the current vector position
        step_size (float): the size fo the step to take

    Output:
        position (list): The new position after taking a step.
    '''
    position = current.copy()
    for i in range(len(position)):
        minimum = max([minmax[i][0], current[i]-step_size])
        maximum = min([minmax[i][1], current[i]+step_size])

        position[i] = rand_in_bounds(minimum, maximum)

    return position

def take_steps(bounds, current, step_size, big_stepsize):
    '''
    Description:
        Calculate the cost of taking a large step of a normal step

    Input:
        bounds (list): the upper and lower bounds for the algorithm
        current (dictionary): current position and cost
        step_size (float): size of a normal step
        big_stepsize (float): size of a large step

    Outpout:
        step (dictionary): the position and cost after taking a normal step
        big_step (dictionary): the position and cost after taking a large step
    '''
    step, big_step = {'vector': [], 'cost': None}, {'vector': [], 'cost': None}

    step['vector'] = take_step(bounds, current['vector'], step_size) 
    step['cost'] = objective_function(step['vector'])

    big_step['vector'] = take_step(bounds, current['vector'], big_stepsize)
    big_step['cost'] = objective_function(big_step['vector'])

    return step, big_step

def search(max_iter, bounds, init_factor, s_factor, l_factor, iter_mult, max_no_impr):
        '''
        Description:
            Handles all of the iterations and searching for optimal solutions

        Input:
            max_iter (int): Number of iterations
            bounds (list): upper and lower bounds for the problem
            init_factor (float): initialize the step size
            s_factor (flaot): take a small step
            l_factor (float): take a large step
            iter_mult (float): Determines when to take a large step
            max_no_improv (int): Determines when to jump to a new location

        Output:
            current (dictionary): contains the lowest cost and associated vectors for all the iterations.
        '''
        step_size = (bounds[0][1]-bounds[0][0])*init_factor
        current, count = {'vector': [], 'cost': None}, 0

        current['vector'] = random_vector(bounds)
        current['cost'] = objective_function(current['vector'])

        for iters in range(max_iter):
            big_stepsize = large_step_size(iters, step_size, s_factor, l_factor, iter_mult)
            step, big_step = take_steps(bounds, current, step_size, big_stepsize)

            if (step['cost'] <= current['cost']) or (big_step['cost'] <= current['cost']):
                if big_step['cost'] <= step['cost']:
                    step_size, current = big_stepsize, big_step.copy()
                else:
                    current = step.copy()
                count = 0
            else:
                count += 1
                if count >= max_no_impr:
                    count, step = 0, (step_size/s_factor)
            print('Iteration: {0}; Cost: {1}'.format(iters, current['cost']))

        return current

In [ ]:
problem_size = 2
bounds = [[-5, 5] for i in range(problem_size)]
max_iter = 1000
init_factor = 0.05
s_factor = 1.3
l_factor = 3.0
iter_mult = 10
max_no_impr = 30

best = search(max_iter, bounds, init_factor, s_factor, l_factor, iter_mult, max_no_impr)
print('******** Done ********')
print("Best Solution is: Cost: {0}; Vector: {1}".format(best['vector'], best['cost']))